# Project Proposal

In [5]:
# Loading Libraries, Remember to run this cell!
library(tidyverse)
library(repr)
library(tidymodels)

options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Warning message:
“package ‘tidymodels’ was built under R version 4.0.2”
── Attaching packages ────────────────────────────────────── tidymodels 0.1.1 ──

✔ broom     0.7.0      ✔ recipes   0.1.13
✔ dials     0.0.9      ✔ rsample   0.0.7 
✔ infer     0.5.4      ✔ tune      0.1.1 
✔ modeldata 0.0.2      ✔ workflows 0.2.0 
✔ parsnip   0.1.3    

### Data Analysis

In [51]:
# Loading in the data
player_stats <- read_csv("https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS")
colnames(player_stats) <- make.names(colnames(player_stats))
#   Remove unnecessary columns
player_stats <- player_stats %>%
    select(Age:Peak.Elo.Rating, -Country, -Wikipedia, -Current.Rank,-Name, -Seasons, -Prize.Money, -Active, -Favorite.Surface, -Best.Elo.Rank, Peak.Elo.Rating)
colnames(player_stats) <- make.names(colnames(player_stats))
#   Mutating columns with as_factor()    
player_stats <- player_stats %>%
    mutate(Plays = as_factor(Plays)) %>%
    mutate(Backhand = as_factor(Backhand)) %>%
#   Cleaning up columns with string values
    mutate(Height = strtoi(str_remove(Height, " cm"))) %>%
    mutate(Age = strtoi(substr(Age, 0, 2))) %>%
    mutate(Best.Rank = strtoi(substr(Best.Rank, 1, nchar(Best.Rank) - 13)))
player_stats

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  `Turned Pro` = col_double(),
  Seasons = col_double(),
  Titles = col_double(),
  `Best Season` = col_double(),
  Retired = col_double(),
  Masters = col_double(),
  `Grand Slams` = col_double(),
  `Davis Cups` = col_double(),
  `Team Cups` = col_double(),
  Olympics = col_double(),
  `Weeks at No. 1` = col_double(),
  `Tour Finals` = col_double()
)

See spec(...) for full column specifications.



Age,Plays,Best.Rank,Backhand,Height,Turned.Pro,Current.Elo.Rank,Peak.Elo.Rating
<int>,<fct>,<int>,<fct>,<int>,<dbl>,<chr>,<chr>
26,Right-handed,363,NA,NA,NA,NA,NA
18,Left-handed,316,Two-handed,NA,NA,NA,NA
32,Right-handed,44,Two-handed,185,2005,144 (1764),1886 (06-02-2012)
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
23,Left-handed,342,NA,NA,NA,NA,NA
24,NA,419,NA,NA,NA,NA,NA
22,Left-handed,408,Two-handed,NA,NA,NA,NA
